# Generacion de Datasets utilizando RLHF para fine tunning

## Procedimiento

- Recoleccion de Datos: Obtener Informacion mediante documentos de textos seleccionables (no escaneados), mediante web, redes sociales, scrapping,etc
- Extraer Texto: Usa PyMuPDF para extraer texto de PDFs no escaneados, procesando en paralelo con ProcessPoolExecutor para manejar grandes volúmenes (>100, >10,000).
- Limpiar Datos: Elimina espacios múltiples y caracteres no deseados con expresiones regulares, asegurando texto coherente.
- Validar: Verifica que los fragmentos extraídos no estén vacíos y tengan una longitud mínima (por ejemplo, 50 caracteres).
- Generar Conversaciones: Divide el texto en párrafos, usa un modelo como meta-llama/Llama-3.2-1B-Instruct para generar diálogos conversacionales específicos al contenido, con al menos 4 intercambios por conversación, en formato {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}], "topic": "..."}.
- Estructurar Dataset: Guarda las conversaciones en archivos JSONL con dos columnas: "messages" y "topic".
Cargar y Subir: Carga el dataset con load_dataset("json", data_files="path/*.jsonl") y súbelo a un repositorio privado en Hugging Face Hub con push_to_hub("username/dataset_name", private=True).

In [1]:
%pip install --upgrade pymupdf nltk ollama openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.8/785.8 kB 187.4 MB/s  0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.1
    Uninstalling openai-1.99.1:━━━━━━━━━━━━━━━━━ 0/2 [openai]
      Successfully uninstalled openai-1.99.1━━━━ 0/2 [openai]
  Attempting uninstall: ollama━━━━━━━━━━━━━━━━━━ 0/2 [openai]
    Found existing installation: ollama 0.5.10m━━━━━━━━━━━━━━━━━━━ 1/2 [ollama]
    Uninstalling ollama-0.5.1:90m╺━━━━━━━━━━━━━━━━━━━ 1/2 [ollama]
      Successfully uninstalled ollama-0.5.1━━━━━━━━━━━━━━━━━━━ 1/2 [ollama]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ollama]2m1/2 [ollama]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

torch.cuda.is_available()

True

In [1]:
import os

OLLAMA_MODEL = "llama3.1:8b"
TEMPERATURE = 0.7  # Equilibrio entre creatividad y coherencia
TOP_P = 0.9  # Filtrado de núcleo para diversidad
MIN_CONVERSATION_LENGTH = 3 

# Constantes
MIN_FRAGMENT_LENGTH = 500
MAX_FRAGMENT_LENGTH = 2000
REPEAT_THRESHOLD = 0.3
MIN_BLOCK_LENGTH = 30  # Reducido de 50 a 30 para incluir bloques más cortos

### Recoleccion de Datos

In [2]:
from pathlib import Path

# Verificar si la carpeta existe

def get_files():
    folder_url = "./docs"
    folder = Path(folder_url)


    if not folder.exists() or not folder.is_dir():
        print("Invalid Folder")
        
    # Obtener todos los archivos de la carpeta
    files = [f for f in folder.rglob("*") if f.is_file()]

    return files

### Extraccion de Texto y Limpieza de Datos

In [3]:
import pymupdf
import re
import os
import hashlib
from collections import Counter



def clean_text(text, repeated_blocks=None):
    """Limpia el texto, eliminando caracteres repetitivos, texto redundante y caracteres no deseados."""
    # Eliminar caracteres de control no deseados (excepto \n)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    # Eliminar patrones repetitivos como ----, ...., ****
    text = re.sub(r'([^\w\s])\1{2,}|\s*[.]{3,}\s*', '', text)
    # Normalizar espacios múltiples
    text = re.sub(r'[ \t]+', ' ', text)
    # Eliminar espacios al inicio y final de cada línea
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    # Filtrar líneas duplicadas, números, correos y contenido administrativo
    seen_lines = set()
    unique_lines = []
    for line in lines:
        if line not in seen_lines and \
           not re.match(r'^\d+$', line) and \
           not re.match(r'.*@(.*\.)+.*', line) and \
           not re.match(r'^(Tel|Fax|E-mail|www\.).*', line, re.IGNORECASE) and \
           (repeated_blocks is None or line not in repeated_blocks):
            unique_lines.append(line)
            seen_lines.add(line)
    return '\n'.join(unique_lines)

def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de un PDF en una sola pasada, preservando el texto del inicio de la página,
    eliminando encabezados, pies de página y contenido irrelevante,
    dividiendo en fragmentos de 500 a 2000 caracteres con metadata corregida.
    """
    try:
        doc = pymupdf.open(pdf_path)
        total_pages = len(doc)
        chunks = []
        current_chunk = []  # Lista de (párrafo, página)
        current_chunk_length = 0
        filename = os.path.basename(pdf_path)
        block_counter = Counter()

        for page_number in range(1, total_pages + 1):
            page = doc[page_number - 1]
            page_height = page.rect.height
            blocks = page.get_text("blocks")
            page_text = []

            # Procesar bloques y filtrar encabezados/pies de página
            for block in blocks:
                text = block[4]
                y0, y1 = block[1], block[3]
                # Excluir pies de página (parte inferior de la página)
                if y1 > 0.95 * page_height:  # Relajado de 0.95 a 0.9
                    continue
                # Excluir encabezados solo si son repetitivos
                if y0 < 0.05 * page_height:  # Relajado de 0.05 a 0.1
                    block_counter[text] += 1
                    if block_counter[text] > total_pages * REPEAT_THRESHOLD:
                        continue
                if text and len(text) >= MIN_BLOCK_LENGTH:
                    block_counter[text] += 1
                    page_text.append((text, page_number))

            # Si no hay texto válido en la página, continuar
            if not page_text:
                continue

            # Acumular párrafos con su número de página
            for paragraph, page in page_text:
                current_chunk.append((paragraph, page))
                current_chunk_length += len(paragraph) + 2  # +2 por "\n\n"

                # Si el fragmento alcanza la longitud mínima, procesarlo
                if current_chunk_length >= MIN_FRAGMENT_LENGTH:
                    chunk_text = "\n\n".join(p for p, _ in current_chunk)
                    cleaned_chunk = clean_text(chunk_text, None)
                    cleaned_length = len(cleaned_chunk)

                    # Obtener el rango de páginas del fragmento
                    page_numbers = sorted(set(page for _, page in current_chunk))
                    start_page = page_numbers[0] if page_numbers else page_number
                    end_page = page_numbers[-1] if page_numbers else page_number

                    # Dividir fragmentos largos
                    while cleaned_length > MAX_FRAGMENT_LENGTH:
                        sub_chunk = cleaned_chunk[:MAX_FRAGMENT_LENGTH]
                        last_paragraph_end = sub_chunk.rfind("\n\n")
                        if last_paragraph_end == -1:
                            last_paragraph_end = MAX_FRAGMENT_LENGTH
                        chunk_to_add = cleaned_chunk[:last_paragraph_end].strip()

                        # Calcular el número de páginas para el subfragmento
                        chars_so_far = 0
                        sub_chunk_pages = []
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far <= last_paragraph_end:
                                sub_chunk_pages.append(page)
                            else:
                                break
                        sub_start_page = min(sub_chunk_pages) if sub_chunk_pages else page_number
                        sub_end_page = max(sub_chunk_pages) if sub_chunk_pages else page_number

                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {len(chunk_to_add)} | "
                            f"PAGES: {sub_start_page}-{sub_end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + chunk_to_add, hashlib.md5(chunk_to_add.encode()).hexdigest()))
                        cleaned_chunk = cleaned_chunk[last_paragraph_end:].strip()
                        cleaned_length = len(cleaned_chunk)

                        # Actualizar current_chunk para los párrafos restantes
                        remaining_chunk = []
                        chars_so_far = 0
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far > last_paragraph_end:
                                remaining_chunk.append((paragraph, page))
                        current_chunk = remaining_chunk
                        current_chunk_length = cleaned_length
                        page_numbers = sorted(set(page for _, page in current_chunk))
                        start_page = page_numbers[0] if page_numbers else page_number

                    # Añadir el fragmento completo
                    if cleaned_length >= MIN_FRAGMENT_LENGTH:
                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                            f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))
                        current_chunk = []
                        current_chunk_length = 0

                    else:
                        current_chunk = [(cleaned_chunk, page_numbers[-1])] if page_numbers else []
                        current_chunk_length = cleaned_length

        # Añadir el fragmento final si cumple con la longitud mínima
        if current_chunk and current_chunk_length >= MIN_FRAGMENT_LENGTH:
            chunk_text = "\n\n".join(p for p, _ in current_chunk)
            cleaned_chunk = clean_text(chunk_text, None)
            cleaned_length = len(cleaned_chunk)
            if cleaned_length >= MIN_FRAGMENT_LENGTH:
                page_numbers = sorted(set(page for _, page in current_chunk))
                start_page = page_numbers[0] if page_numbers else total_pages
                end_page = page_numbers[-1] if page_numbers else total_pages
                metadata = (
                    f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                    f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                )
                chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))

        doc.close()

        # Filtrar bloques repetitivos y duplicados
        repeated_blocks = {text for text, count in block_counter.items() if count > total_pages * REPEAT_THRESHOLD}
        final_chunks = []
        seen_hashes = set()

        for chunk, chunk_hash in chunks:
            chunk_text = '\n'.join(line for line in chunk.splitlines() if not line.startswith('#'))
            cleaned_chunk = clean_text(chunk_text, repeated_blocks)
            if len(cleaned_chunk) >= MIN_FRAGMENT_LENGTH and chunk_hash not in seen_hashes:
                # Actualizar la longitud en la metadata después de la limpieza final
                metadata_lines = chunk.splitlines()[0]
                metadata = re.sub(r'CHARACTERS: \d+', f'CHARACTERS: {len(cleaned_chunk)}', metadata_lines)
                final_chunks.append(f"{metadata}\n\n{cleaned_chunk}")
                seen_hashes.add(chunk_hash)

        return final_chunks if final_chunks else None
    except Exception as e:
        print(f"Error procesando {pdf_path}: {e}")
        return None

### Generar Conversaciones

In [4]:
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
load_dotenv()

class Message(BaseModel):
    role: str = Field(..., pattern="^(user|assistant)$")
    content: str

class Conversation(BaseModel):
    messages: List[Message] = Field(..., min_items=MIN_CONVERSATION_LENGTH)
    topic: str

In [49]:
def get_prompts(fragment: str):
  INSTRUCTIONS = """
  Eres un generador de conversaciones optimizadas para entrenamiento supervisado (SFT) con técnicas avanzadas de prompt engineering. 
Tu objetivo es producir diálogos profesionales, coherentes y progresivos basados en fragmentos de texto, orientados al ámbito laboral. 
Cada conversación debe tener continuidad entre turnos y mantener un estilo claro, formal pero natural.

<instrucciones>

1. **Formato de salida**:
- Devuelve un JSON válido con:
  - `messages`: Lista con { "role": "user" | "assistant", "content": "..." }
  - `topic`: cadena breve (máx. 50 caracteres)
- **No incluir rol "system"** en la salida final.

2. **Estructura de la conversación**:
- 3 a 4 intercambios (mínimo 4 y máximo 8 mensajes).
- Siempre empieza con "user" y alterna roles.
- Cada mensaje del usuario debe partir del contexto previo y avanzar el tema.
- Respuestas del asistente con razonamiento paso a paso (chain-of-thought) y ejemplos prácticos.
- Usa Markdown para estructurar la respuesta: títulos, listas, negritas, etc.

3. **Tono y estilo**:
- Profesional y aplicable en contextos de trabajo (consultoría, gestión de proyectos, análisis técnico, comunicación corporativa).
- Lenguaje formal pero natural, evitando jerga innecesaria.
- Respuestas concisas pero completas, con enfoque en resultados y acciones concretas.

4. **Patrones de pregunta** (adaptados a entornos laborales):
- "Me gustaría implementar {tema} en la empresa. ¿Qué pasos sugieres?"
- "Basado en {contexto}, propón un plan detallado para {objetivo}."
- "Genera un cronograma para {proyecto}, asignando responsables."
- "Identifica riesgos y métricas para {iniciativa}."
- "Crea un documento de recomendaciones sobre {tema}."
- "Evalúa ventajas y desventajas de {opción1} frente a {opción2}."
- "Desglosa en fases el proceso para {tarea}."
- "Propón mejoras para optimizar {área o proceso}."
- "Dame indicadores clave para medir {resultado}."
- "Elabora un resumen ejecutivo sobre {tema} para la gerencia."

5. **Criterios de calidad**:
- Coherencia en todos los turnos.
- Uso de información previa en las respuestas siguientes.
- Evitar repeticiones innecesarias.
- Claridad en la explicación y aplicabilidad práctica.


<PROMPT_PATTERNS>
- "Me intriga {tema}. ¿Puedes explicármelo detalladamente?"
- "En pocas palabras, resúmeme cómo funciona {tema}."
- "Crea una carta formal sobre {tema} dirigida a {destinatario}."
- "Basado en {contexto}, explícame qué significa {tema}."
- "Quiero mejorar en {tema}. ¿Qué consejos prácticos me das?"
- "Agrega una sección sobre {tema} a este informe: {contexto}."
- "No entiendo {tema}. ¿Puedes desglosarlo en pasos simples?"
- "Imagina que soy nuevo en {tema}. ¿Cómo me lo explicarías?"
- "Podrías darme un resumen sobre {tema} y sus implicaciones?"
- "Estoy preocupado por {tema}. ¿Qué debería saber al respecto?"
- "Explícame {tema} como si fuera un niño de 10 años."
- "Redacta un correo informal sobre {tema} para un colega."
</PROMPT_PATTERNS>
"""

  BASE_EXAMPLE = """"

**Ejemplo n — Gestión Documental**
<fragment>
Implementar un sistema de gestión documental para mejorar la búsqueda y control de versiones.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Necesito un plan para implementar un sistema de gestión documental en la empresa. Tenemos problemas con la búsqueda de archivos y duplicación de versiones."
    },
    {
      "role": "assistant",
      "content": "Claro. Vamos a estructurarlo paso a paso:\\n\\n**1. Análisis inicial**\\n- Auditar los flujos actuales de manejo documental.\\n- Identificar carpetas duplicadas y problemas de control de versiones.\\n\\n**2. Selección de plataforma**\\n- Comparar opciones como SharePoint, Alfresco o Google Workspace.\\n- Priorizar integración, seguridad y facilidad de búsqueda.\\n\\n**3. Migración y normalización**\\n- Definir nomenclatura estándar.\\n- Centralizar repositorios.\\n\\n**4. Capacitación y seguimiento**\\n- Formación en la plataforma elegida.\\n- Revisión mensual para detectar desviaciones."
    },
    {
      "role": "user",
      "content": "Sí, dame el cronograma y asigna responsables para un equipo de 5 personas en administración."
    },
    {
      "role": "assistant",
      "content": "**Cronograma — Implementación Gestión Documental**\\n\\n**Semana 1-2 — Auditoría**\\n- Responsable: Laura (Jefa de administración)\\n\\n**Semana 3 — Selección de plataforma**\\n- Responsable: Jorge (Analista de sistemas)\\n\\n**Semana 4-6 — Migración inicial**\\n- Responsables: Ana y Ricardo\\n\\n**Semana 7 — Capacitación**\\n- Responsable: Mariana\\n\\n**Semana 8-9 — Seguimiento**\\n- Responsable: Laura"
    },
    {
      "role": "user",
      "content": "Perfecto. ¿Puedes incluir métricas para evaluar si la implementación fue exitosa?"
    },
    {
      "role": "assistant",
      "content": "**Métricas de éxito**:\\n- Tiempo de búsqueda reducido en 50%.\\n- Menos del 5% de duplicados.\\n- 90% de cumplimiento en nomenclatura.\\n- 95% de adopción entre usuarios.\\n- Satisfacción ≥ 8/10 en encuestas internas."
    }
  ],
  "topic": "Implementación de gestión documental"
}
</response>

  """

  EXAMPLES = """
<ejemplos>

Ejemplo Base:
%s

Otros Ejemplos:

**Ejemplo 1 — Técnico (dispersión de luz, patrón: explicación detallada, fragmento largo)**:
<fragment>
La dispersión de Rayleigh explica por qué el cielo aparece azul durante el día. Las partículas en la atmósfera dispersan preferentemente las longitudes de onda cortas, como el azul, debido a su interacción con la luz solar. Durante el atardecer, el cielo adquiere tonos rojos porque la luz atraviesa más atmósfera, dispersando el azul y dejando predominar las longitudes de onda largas. Este fenómeno depende de la densidad atmosférica y el ángulo de incidencia de la luz solar.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Me intriga por qué el cielo es azul de día y rojo al atardecer. ¿Puedes explicármelo detalladamente?"
    },
    {
      "role": "assistant",
      "content": "# Dispersión de Rayleigh y el Color del Cielo\n\nPara entender por qué el cielo cambia de color, desglosemos el fenómeno paso a paso:\n\n1. **Composición de la luz solar**: La luz del sol parece blanca, pero está formada por todas las longitudes de onda visibles, desde el rojo (largo) hasta el violeta (corto).\n\n2. **Dispersión de Rayleigh**: En la atmósfera, las partículas (como moléculas de nitrógeno y oxígeno) dispersan la luz. La dispersión es más intensa para longitudes de onda cortas, como el azul, porque es inversamente proporcional a la **cuarta potencia** de la longitud de onda. Esto significa que el azul se esparce mucho más que el rojo.\n\n3. **Cielo diurno**: Durante el día, la luz solar atraviesa una capa relativamente delgada de atmósfera. El azul se dispersa en todas direcciones, haciendo que el cielo se vea azul. Aunque el violeta se dispersa más, nuestros ojos son menos sensibles a él, y parte es absorbida, por lo que el azul domina.\n\n4. **Atardecer**: Al atardecer, el sol está más bajo, y la luz atraviesa una capa más gruesa de atmósfera. Esto dispersa casi todo el azul, permitiendo que las longitudes de onda largas (rojo, naranja) lleguen a nuestros ojos, dando tonos cálidos.\n\n5. **Factores adicionales**: La densidad atmosférica y el ángulo de incidencia afectan la intensidad del fenómeno. Por ejemplo, más partículas (como polvo) pueden intensificar los colores del atardecer.\n\nEn resumen, la dispersión de Rayleigh explica el azul diurno y los tonos rojos del atardecer, influenciada por la física de la luz y la geometría de la Tierra."
    },
    {
      "role": "user",
      "content": "¿Por qué a veces el cielo se ve más violeta en ciertos atardeceres?"
    },
    {
      "role": "assistant",
      "content": "A veces, el cielo puede parecer violeta en el atardecer por la presencia de partículas adicionales, como polvo o contaminación, que dispersan más longitudes de onda cortas, incluyendo el violeta. Además, si la atmósfera está muy limpia y el sol está en un ángulo específico, el violeta puede ser más visible porque menos se absorbe. Sin embargo, nuestros ojos suelen percibir más el azul, así que el violeta es menos común."
    }
  ],
  "topic": "Dispersión de luz y color del cielo"
}
</response>

**Ejemplo 2 — Técnico (dispersión de luz, patrón: resumen breve, fragmento corto)**:
<fragment>
La dispersión de Rayleigh hace que el cielo sea azul al dispersar más las longitudes de onda cortas.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "En pocas palabras, resúmeme cómo funciona la dispersión de luz en el cielo."
    },
    {
      "role": "assistant",
      "content": "La dispersión de Rayleigh ocurre cuando las partículas atmosféricas dispersan más las longitudes de onda cortas, como el azul, haciendo que el cielo se vea azul durante el día."
    }
  ],
  "topic": "Dispersión de luz en el cielo"
}
</response>

**Ejemplo 3 — Contextual (política monetaria, patrón: carta formal, fragmento largo)**:
<fragment>
Subir las tasas de interés es una herramienta del banco central para controlar la inflación. Al encarecer los préstamos, se reduce el consumo y la inversión, lo que desacelera la economía y disminuye la presión sobre los precios. Sin embargo, un aumento excesivo puede llevar a una recesión, afectando el empleo y el crecimiento económico. Los bancos centrales deben monitorear indicadores como el consumo y el desempleo para ajustar las tasas cuidadosamente.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Crea una carta formal sobre el impacto de las tasas de interés, dirigida al director de mi banco."
    },
    {
      "role": "assistant",
      "content": "**Asunto**: Preocupación por el Impacto de las Tasas de Interés\n\nEstimado Director,\n\nMe dirijo a usted para expresar mi preocupación sobre el reciente aumento de las tasas de interés y su impacto en los clientes. Entiendo que el banco central sube las tasas para controlar la inflación, encareciendo los préstamos, lo que reduce el consumo y la presión sobre los precios. Sin embargo, esto afecta directamente a los consumidores, elevando el costo de hipotecas y créditos, y podría desencadenar una recesión si las tasas suben demasiado.\n\nSolicito información sobre cómo el banco planea apoyar a los clientes en este contexto, como opciones de refinanciamiento o asesoramiento financiero. Espero su respuesta en un plazo de 7 días hábiles.\n\nAtentamente,  \n[Nombre del Cliente]"
    },
    {
      "role": "user",
      "content": "Agrega una mención sobre el impacto en las pequeñas empresas."
    },
    {
      "role": "assistant",
      "content": "**Asunto**: Preocupación por el Impacto de las Tasas de Interés\n\nEstimado Director,\n\nMe dirijo a usted para expresar mi preocupación sobre el aumento de las tasas de interés. Estas medidas, destinadas a controlar la inflación al encarecer los préstamos, reducen el consumo y la inversión, afectando tanto a los consumidores como a las pequeñas empresas, que enfrentan mayores costos de financiamiento. Esto podría desencadenar una recesión, impactando el empleo y el crecimiento económico.\n\nSolicito información sobre cómo el banco apoyará a los clientes, incluyendo a pequeñas empresas, con opciones como refinanciamiento o asesoramiento financiero. Espero su respuesta en 7 días hábiles.\n\nAtentamente,  \n[Nombre del Cliente]"
    }
  ],
  "topic": "Impacto de tasas de interés"
}
</response>

**Ejemplo 4 — Incoherente (fragmento ambiguo, patrón: contexto adicional, fragmento corto)**:
<fragment>
...fluctuaciones de energía... inestabilidad...
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Basado en problemas con el parpadeo de luces en mi casa, explícame qué significan fluctuaciones de energía."
    },
    {
      "role": "assistant",
      "content": "Las fluctuaciones de energía son cambios en el voltaje eléctrico, causando inestabilidad como el parpadeo de luces. Pueden deberse a cableado defectuoso o sobrecarga en la red. Usa un regulador de voltaje o consulta a un electricista."
    }
  ],
  "topic": "Fluctuaciones de energía"
}
</response>

**Ejemplo 5 — Casual (salud visual, patrón: consejos prácticos, fragmento largo)**:
<fragment>
La exposición prolongada a la luz azul de pantallas puede interrumpir la producción de melatonina, afectando el ciclo de sueño. Esto ocurre porque la luz azul engaña al cerebro, haciéndole creer que es de día, lo que retrasa el inicio del sueño. También puede causar fatiga visual y molestias como sequedad ocular.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Quiero mejorar mi sueño porque paso mucho tiempo frente a pantallas. ¿Qué consejos prácticos me das?"
    },
    {
      "role": "assistant",
      "content": "# Mejorar el Sueño y Proteger la Vista frente a Pantallas\n\nLa luz azul de las pantallas puede afectar tu sueño y ojos, pero hay formas prácticas de mitigar esto. Aquí van algunos consejos:\n\n- **Usa el modo nocturno**: Activa el filtro de luz azul en tus dispositivos para reducir su emisión, especialmente por la noche.\n- **Evita pantallas antes de dormir**: Intenta no usar dispositivos al menos 1 hora antes de acostarte para que la melatonina, la hormona del sueño, se produzca normalmente.\n- **Aplica la regla 20-20-20**: Cada 20 minutos, mira algo a 20 pies de distancia durante 20 segundos para descansar tus ojos y evitar fatiga visual.\n- **Considera gafas con filtro**: Si trabajas mucho con pantallas, unas gafas con filtro de luz azul pueden reducir molestias como sequedad ocular.\n- **Mantén un ambiente adecuado**: Usa iluminación suave en tu espacio de trabajo para reducir el contraste con la pantalla.\n\n*Por qué funciona*: La luz azul engaña al cerebro, haciéndole creer que es de día, lo que inhibe la melatonina y retrasa el sueño. Estos pasos ayudan a proteger tu ciclo de sueño y la salud de tus ojos."
    },
    {
      "role": "user",
      "content": "¿Qué hago si no puedo evitar las pantallas por trabajo?"
    },
    {
      "role": "assistant",
      "content": "Si no puedes evitar pantallas, refuerza las medidas: usa siempre el modo nocturno, haz pausas frecuentes con la regla 20-20-20, y considera gafas con filtro de luz azul. También ajusta el brillo de la pantalla para que sea similar al de tu entorno y evita trabajar en completa oscuridad."
    }
  ],
  "topic": "Luz azul y sueño"
}
</response>

**Ejemplo 6 — Contextual (criptografía, patrón: informe, fragmento largo)**:
<fragment>
La criptografía de clave pública utiliza un par de claves: una pública para cifrar y una privada para descifrar. Esto asegura que solo el destinatario con la clave privada pueda acceder al mensaje, garantizando confidencialidad y autenticidad. Es ampliamente usada en transacciones seguras en línea.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Agrega una sección sobre criptografía de clave pública a este informe: 'Seguridad en transacciones digitales'."
    },
    {
      "role": "assistant",
      "content": "## Criptografía de Clave Pública en Transacciones Digitales\n\nLa **criptografía de clave pública** es un pilar fundamental para la seguridad en transacciones digitales. Funciona mediante un par de claves:\n\n- **Clave pública**: Se comparte libremente y permite cifrar mensajes, convirtiéndolos en texto ilegible.\n- **Clave privada**: Solo la posee el destinatario y permite descifrar el mensaje.\n\nEste mecanismo asegura:\n\n1. **Confidencialidad**: Solo el destinatario con la clave privada puede leer el mensaje.\n2. **Autenticidad**: Si el remitente firma con su clave privada, el receptor verifica su identidad con la clave pública.\n\n*Ejemplo práctico*: En compras online, la criptografía de clave pública protege los datos de tu tarjeta, asegurando que solo el vendedor autorizado los descifre. Es esencial para protocolos como HTTPS y sistemas de pago seguro."
    }
  ],
  "topic": "Criptografía en transacciones"
}
</response>

**Ejemplo 7 — Técnico (cambio climático, patrón: pasos simples, fragmento corto)**:
<fragment>
El cambio climático aumenta las temperaturas globales debido a los gases de efecto invernadero.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "No entiendo el cambio climático. ¿Puedes desglosarlo en pasos simples?"
    },
    {
      "role": "assistant",
      "content": "1. Quemamos combustibles como carbón y petróleo.\n2. Esto libera gases de efecto invernadero, como el dióxido de carbono.\n3. Los gases atrapan calor en la atmósfera.\n4. El planeta se calienta, causando cambio climático."
    }
  ],
  "topic": "Cambio climático explicado"
}
</response>

**Ejemplo 8 — Casual (nutrición, patrón: explicación para principiantes, fragmento largo)**:
<fragment>
El consumo excesivo de azúcar puede aumentar el riesgo de diabetes tipo 2 y obesidad. Reducir la ingesta de bebidas azucaradas y alimentos procesados es clave para una dieta saludable. Los azúcares añadidos en productos como refrescos, dulces y salsas comerciales suelen ser los principales culpables.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Imagina que soy nuevo en nutrición. ¿Cómo me explicarías lo del azúcar?"
    },
    {
      "role": "assistant",
      "content": "Piénsalo como gasolina para tu cuerpo: el azúcar te da energía, pero demasiada puede hacerte daño. Hay azúcar natural, como en las frutas, que está bien en moderación. Pero el problema es el **azúcar añadido**, que encuentras en refrescos, dulces o salsas. Comer mucho de esto puede subir tu peso y aumentar el riesgo de diabetes tipo 2, porque tu cuerpo lucha para manejar tanto azúcar. Para comer mejor, cambia refrescos por agua, elige snacks como nueces y lee etiquetas para evitar productos con azúcares ocultos."
    }
  ],
  "topic": "Azúcar y salud"
}
</response>

**Ejemplo 9 — Incoherente (fragmento sin contexto, patrón: resumen e implicaciones, fragmento corto)**:
<fragment>
...error crítico en el núcleo...
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Podrías darme un resumen sobre un error crítico en el núcleo y sus implicaciones?"
    },
    {
      "role": "assistant",
      "content": "# Error Crítico en el Núcleo\n\n**Resumen**: Un error crítico en el núcleo es un fallo grave en el sistema operativo, que actúa como el cerebro de tu computadora. Puede ocurrir por hardware defectuoso, software corrupto o malware.\n\n**Implicaciones**:\n- **Pérdida de datos**: Los archivos pueden perderse si el sistema no guarda correctamente.\n- **Interrupciones**: El sistema puede congelarse o reiniciarse.\n- **Costos**: Reparaciones o recuperación de datos pueden ser necesarias.\n\n**Acción**: Usa software de recuperación como TestDisk y haz backups regulares para minimizar riesgos."
    }
  ],
  "topic": "Error crítico en sistema"
}
</response>

**Ejemplo 10 — Contextual (economía, patrón: preocupación, fragmento largo)**:
<fragment>
La inflación persistente puede erosionar el poder adquisitivo, haciendo que los bienes y servicios sean más caros. Los bancos centrales intentan controlarla con políticas monetarias, pero esto puede tener efectos secundarios en el empleo y el crecimiento económico.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Estoy preocupado por la inflación. ¿Qué debería saber al respecto?"
    },
    {
      "role": "assistant",
      "content": "# Entendiendo la Inflación\n\nLa inflación ocurre cuando los precios de bienes y servicios suben, reduciendo tu **poder adquisitivo** (es decir, con el mismo dinero compras menos). Aquí está lo que debes saber:\n\n- **Causas**: Puede ser por alta demanda, costos de producción elevados o políticas monetarias expansivas.\n- **Impacto**: Tus ahorros valen menos, y cosas como alimentos o gasolina se encarecen.\n- **Control**: Los bancos centrales suben tasas de interés para reducir el gasto, pero esto puede ralentizar la economía, afectando empleos.\n\n**Qué hacer**:\n- **Ahorra inteligentemente**: Invierte en activos que superen la inflación, como bonos ajustados.\n- **Presupuesto**: Prioriza gastos esenciales y reduce los no necesarios.\n- **Infórmate**: Sigue las políticas del banco central para anticipar cambios.\n\nHabla con un asesor financiero si quieres estrategias específicas."
    }
  ],
  "topic": "Inflación y poder adquisitivo"
}
</response>

**Ejemplo 11 — Técnico (energía renovable, patrón: explicación para niño, fragmento corto)**:
<fragment>
La energía solar usa paneles para convertir la luz del sol en electricidad.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Explícame la energía solar como si fuera un niño de 10 años."
    },
    {
      "role": "assistant",
      "content": "Imagina que el sol es como una lámpara gigante que brilla todo el día. La **energía solar** usa placas especiales, como espejos mágicos, que atrapan la luz del sol y la convierten en electricidad para encender luces, televisions o cargar tu tablet. ¡Es como usar el poder del sol para hacer cosas cool!"
    }
  ],
  "topic": "Energía solar simplificada"
}
</response>

**Ejemplo 12 — Casual (trabajo remoto, patrón: correo informal, fragmento largo)**:
<fragment>
El trabajo remoto puede aumentar la productividad, pero también presenta desafíos como la falta de interacción social y dificultades para desconectarse del trabajo. Herramientas como Zoom y Slack facilitan la colaboración, pero requieren una buena gestión del tiempo.
</fragment>
<response>
{
  "messages": [
    {
      "role": "user",
      "content": "Redacta un correo informal sobre trabajo remoto para un colega."
    },
    {
      "role": "assistant",
      "content": "**Asunto**: Tips para el Trabajo Remoto\n\nHola [Nombre],\n\nEstuve pensando en cómo sacarle provecho al trabajo desde casa. Puede ser súper productivo porque evitas el tráfico, pero a veces se siente raro sin las charlas en la oficina, ¿no? Herramientas como Zoom o Slack ayudan a mantenernos conectados, pero la clave está en organizarse bien. Intenta poner horarios fijos para trabajar y desconectarte, así no terminas respondiendo correos a medianoche. ¿Tú cómo lo llevas? ¡Cuéntame si tienes algún truco!\n\nSaludos,  \n[Tu Nombre]"
    }
  ],
  "topic": "Trabajo remoto y productividad"
}
</response>

</ejemplos>
"""
  system_examples = EXAMPLES % BASE_EXAMPLE
  system_content = f"{INSTRUCTIONS}\n{system_examples}"
  return [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": fragment
        }
    ]

In [51]:
def validate_genterated_conv(conversation:Conversation):
  # Validar que la conversación tenga al menos un intercambio completo
  if len(conversation.messages) < 2:
      print(f"Error: Conversación inválida, número de mensajes insuficiente: {len(conversation.messages)}")
      return None
  # Si el número de mensajes es impar, eliminar el último (asumiendo que es del usuario)
  if len(conversation.messages) % 2 != 0 and len(conversation.messages) > 2:
      conversation.messages = conversation.messages[:-1]
      
  return conversation

In [52]:
from ollama import AsyncClient

ollama_client = AsyncClient()

async def generate_ollama_conversations(messages):
    try:
        # Llama a Ollama con roles system y user
        response = await ollama_client.chat(
            model=OLLAMA_MODEL,
            messages=messages,
            options={
                "temperature": TEMPERATURE,
                "top_p": TOP_P,
                
                
            },
            format=Conversation.model_json_schema()  # Especifica el esquema JSON
            
        )
        conversation = Conversation.model_validate_json(response.message.content)

        return validate_genterated_conv(conversation)
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        return None


In [53]:
from openai import AsyncClient, RateLimitError

openai_client = AsyncClient(api_key=os.environ.get("OPENAI_API_KEY"))

async def generate_openai_conversations(messages, model="gpt-4o-mini"):
    try:
        response = await openai_client.responses.parse(
            model=model,
            input=messages,
            text_format=Conversation
        )
        conversation = response.output_parsed
        return validate_genterated_conv(conversation) if conversation else None
    except RateLimitError as e:
        print(f"Rate limit hit for {model}: {e}")
        next_model = "gpt-5-nano"
        if next_model != model:
            return await generate_openai_conversations(messages, next_model)
        return None
    except Exception as e:
        print(f"Error generating conversation with OpenAI {model}: {e}")
        return None

In [54]:
from typing import Literal
async def generate_conversation(fragment:str, provider: Literal["ollama", "openai"]):
    """
    Genera una conversación estructurada en formato JSON usando Ollama, basada en un fragmento de texto.
    Optimizado para múltiples iteraciones en la generación de datasets para fine-tuning.
    """
    if not fragment or len(fragment) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Fragmento demasiado corto ({len(fragment)} caracteres).")
        return None

    # Extraer el texto sin la metadata (líneas que comienzan con '#')
    fragment_content = '\n'.join(line for line in fragment.splitlines() if not line.startswith('#')).strip()
    if len(fragment_content) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Contenido útil del fragmento demasiado corto ({len(fragment_content)} caracteres).")
        return None

    # Prompt optimizado para múltiples iteraciones
    messages = get_prompts(fragment_content)

    try:
        if provider=='ollama':
            return await generate_ollama_conversations(messages)
        elif provider=='openai':
            return await generate_openai_conversations(messages)
        else: return None
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        return None

In [55]:
import logging

# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('pdf_processing.log'),
        logging.StreamHandler()
    ]
)

In [56]:
import json
import os
from filelock import FileLock
import asyncio
from typing import Literal


OUTPUT_FOLDER = "data"

class MetadataManager:
    """Clase para manejar archivos de metadatos JSON."""
    def __init__(self, index: int, output_dir: str):
        self.folder = os.path.join(output_dir, "metadata")
        self.metadata_file = os.path.join(self.folder, f"metadata_{index:04d}.json")
        self.lock_file = f"{self.metadata_file}.lock"
        self.index = index
        os.makedirs(self.folder, exist_ok=True)

    def exists(self) -> bool:
        """Verifica si el archivo de metadatos existe."""
        return os.path.exists(self.metadata_file)

    def get(self, param: str):
        """Obtiene chunks, conversations, fileName, num_chunks, num_messages o num_exchanges desde el archivo de metadatos."""
        if not self.exists():
            return [] if param in ["chunks", "conversations"] else 0 if param in ["num_chunks", "num_messages", "num_exchanges"] else ""
        try:
            with FileLock(self.lock_file):
                with open(self.metadata_file, "r", encoding="utf-8") as f:
                    metadata = json.load(f)
                
                updated = False
                if param == "conversations":
                    conversations = metadata.get("conversations", [])
                    for i, conv in enumerate(conversations):
                        if isinstance(conv, dict) and "chunk_index" not in conv:
                            conv["chunk_index"] = i
                            updated = True
                    conversations.sort(key=lambda x: x.get("chunk_index", 0))
                    metadata["conversations"] = conversations
                
                if "num_chunks" not in metadata and "chunks" in metadata:
                    metadata["num_chunks"] = len(metadata.get("chunks", []))
                    updated = True
                
                if "num_exchanges" not in metadata and "conversations" in metadata:
                    metadata["num_exchanges"] = sum(len(conv["messages"]) // 2 for conv in metadata.get("conversations", []))
                    updated = True
                
                if "num_messages" not in metadata and "conversations" in metadata:
                    metadata["num_messages"] = sum(len(conv["messages"]) for conv in metadata.get("conversations", []))
                    updated = True
                    
                if "total_conversations" not in metadata and "conversations" in metadata:
                    metadata["total_conversations"] = len(conversations)
                    updated = True
                
                if updated:
                    with open(self.metadata_file, "w", encoding="utf-8") as f:
                        json.dump(metadata, f, ensure_ascii=False, indent=2)
                
                return metadata.get(param, []) if param in ["chunks", "conversations"] else metadata.get(param, 0 if param in ["num_chunks", "num_messages", "num_exchanges"] else metadata.get(param,""))
        except Exception as e:
            print(f"Error al leer {param} desde {self.metadata_file}: {e}")
            return [] if param in ["chunks", "conversations"] else 0 if param in ["num_chunks", "num_messages", "num_exchanges"] else ""

    def set(self, param: str, value):
        """Establece chunks, conversations, fileName, num_chunks, num_messages o num_exchanges en el archivo de metadatos."""
        try:
            with FileLock(self.lock_file):
                metadata = {"chunks": [], "fileName": "", "conversations": [], "num_chunks": 0, "num_messages": 0, "num_exchanges": 0}
                if self.exists():
                    with open(self.metadata_file, "r", encoding="utf-8") as f:
                        metadata = json.load(f)
                
                metadata[param] = value
                if param == "conversations":
                    metadata["conversations"].sort(key=lambda x: x.get("chunk_index", 0))
                    metadata["num_messages"] = sum(len(conv["messages"]) for conv in metadata["conversations"])
                    metadata["num_exchanges"] = sum(len(conv["messages"]) // 2 for conv in metadata["conversations"])
                    metadata["total_conversations"] = len(metadata["conversations"])
                if param == "chunks":
                    metadata["num_chunks"] = len(value)
                with open(self.metadata_file, "w", encoding="utf-8") as f:
                    json.dump(metadata, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error al escribir {param} en {self.metadata_file}: {e}")

    def append_conversation(self, conversation: dict):
        """Añade una conversación al array de conversaciones en el archivo de metadatos."""
        try:
            with FileLock(self.lock_file):
                metadata = {"chunks": [], "fileName": "", "conversations": [], "num_chunks": 0, "num_messages": 0, "num_exchanges": 0}
                if self.exists():
                    with open(self.metadata_file, "r", encoding="utf-8") as f:
                        metadata = json.load(f)
                
                for i, conv in enumerate(metadata["conversations"]):
                    if isinstance(conv, dict) and "chunk_index" not in conv:
                        conv["chunk_index"] = i
                
                if "num_chunks" not in metadata:
                    metadata["num_chunks"] = len(metadata.get("chunks", []))
                
                if "num_exchanges" not in metadata:
                    metadata["num_exchanges"] = sum(len(conv["messages"]) // 2 for conv in metadata.get("conversations", []))
                
                if "num_messages" not in metadata:
                    metadata["num_messages"] = sum(len(conv["messages"]) for conv in metadata.get("conversations", []))
                
                metadata["conversations"].append(conversation)
                metadata["conversations"].sort(key=lambda x: x.get("chunk_index", 0))
                metadata["num_messages"] = sum(len(conv["messages"]) for conv in metadata["conversations"])
                metadata["num_exchanges"] = sum(len(conv["messages"]) // 2 for conv in metadata["conversations"])
                metadata["total_conversations"] = len(metadata["conversations"])
                
                with open(self.metadata_file, "w", encoding="utf-8") as f:
                    json.dump(metadata, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error al añadir conversación a {self.metadata_file}: {e}")

def get_text_from_pdf(index: int, pdf_path: str, output_dir: str) -> list:
    """Obtiene el texto de un PDF desde la caché (JSON) o lo extrae si no existe."""
    metadata_manager = MetadataManager(index, output_dir)
    
    if metadata_manager.exists():
        chunks = metadata_manager.get("chunks")
        file_name = metadata_manager.get("fileName")
        if chunks and file_name == os.path.basename(pdf_path):
            return chunks
    
    try:
        pages_text = extract_text_from_pdf(pdf_path)
        if pages_text:
            metadata_manager.set("chunks", pages_text)
            metadata_manager.set("fileName", os.path.basename(pdf_path))
            metadata_manager.set("conversations", [])
        return pages_text
    except Exception as e:
        print(f"Error al extraer texto del PDF {pdf_path}: {e}")
        return []
    
def get_conv_from_jsonl(index, output_dir):
    metadata_manager = MetadataManager(index, output_dir)
    jsonl_file = os.path.join(output_dir, OUTPUT_FOLDER, f"pdf_{index:04d}.jsonl")
    if os.path.exists(jsonl_file):
        try:
            chunks = metadata_manager.get("chunks")
            with open(jsonl_file, "r", encoding="utf-8") as f:
                conversations = []
                for i, line in enumerate(f):
                    if line.strip():
                        conv = json.loads(line)
                        conv["source_chunk"] = chunks[i]
                        conv["chunk_index"] = i
                        conversations.append(conv)
                conversations.sort(key=lambda x: x.get("chunk_index", 0))
                metadata_manager.set("conversations", conversations)
                metadata_manager.set("num_messages", sum(len(conv["messages"]) for conv in conversations))
                metadata_manager.set("num_exchanges", sum(len(conv["messages"]) // 2 for conv in conversations))
                metadata_manager.set("total_conversations", len(conversations))
                return conversations
        except Exception as e:
            print(f"Error al leer conversaciones desde {jsonl_file}: {e}")
    return []

async def get_conversation_from_chunk(index: int, output_dir: str, chunk: str, chunk_index: int, provider: Literal["ollama", "openai"]):
    """Obtiene o genera una conversación para un fragmento de texto."""
    metadata_manager = MetadataManager(index, output_dir)
    existing_conversations = metadata_manager.get("conversations")
    
    if not existing_conversations:
        existing_conversations = get_conv_from_jsonl(index, output_dir)
    
    for conv in existing_conversations:
        if isinstance(conv, dict) and conv.get("source_chunk") == chunk and conv.get("chunk_index") == chunk_index:
            return conv
    try:
        conversation = await generate_conversation(chunk, provider)
        if conversation:
            conv_dict = conversation.model_dump()
            conv_dict["source_chunk"] = chunk
            conv_dict["chunk_index"] = chunk_index
            conv_dict["provider"]= provider
            from datetime import datetime
            conv_dict["created_at"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            metadata_manager.append_conversation(conv_dict)
            return conv_dict
        return None
    except Exception as e:
        print(f"Error al generar conversación para chunk_index {chunk_index} en PDF #{index} con {provider}: {e}")
        return None

async def get_conv(index: int, output_dir: str, chunk: str, chunk_index: int, provider: Literal["ollama", "openai"]):
    result = await get_conversation_from_chunk(index, output_dir, chunk, chunk_index, provider)
    return result if result is not None else []

async def process_pdf(index: int, pdf_path: str, output_dir: str):
    """Procesa un PDF, genera conversaciones y las guarda en JSONL."""
    logger = logging.getLogger()
    logger.info(f"[PDF {index}] Procesando PDF: {Path(pdf_path).name}")
    try:
        pages_text = get_text_from_pdf(index, pdf_path, output_dir)
        
        if not pages_text:
            logger.warning(f"[PDF {index}] No se encontraron fragmentos de texto")
            return
        
        data_dir = os.path.join(output_dir, OUTPUT_FOLDER)
        os.makedirs(data_dir, exist_ok=True)
        jsonl_file = os.path.join(data_dir, f"pdf_{index:04d}.jsonl")
        
        tasks = [(i, fragment) for i, fragment in enumerate(pages_text) if len(fragment) > 20]
        if not tasks:
            logger.warning(f"[PDF {index}] No hay fragmentos válidos para procesar")
            return
        
        logger.info(f"[PDF {index}] Procesando {len(tasks)} fragmentos")
        
        # Dividir tareas entre providers (50% openai, 50% ollama)
        total_tasks = len(tasks)
        half_tasks = total_tasks // 2
        openai_tasks = [get_conv(index, output_dir, fragment, i, "openai") for i, fragment in tasks[:half_tasks]]
        ollama_tasks = [get_conv(index, output_dir, fragment, i, "ollama") for i, fragment in tasks[half_tasks:]]
        
        # Ejecutar tareas en paralelo con límite de concurrencia
        from asyncio import Semaphore
        async def limited_gather(tasks, limit=10):
            semaphore = Semaphore(limit)
            async def sem_task(task):
                async with semaphore:
                    return await task
            return await asyncio.gather(*[sem_task(task) for task in tasks], return_exceptions=True)
        
        task_results = await limited_gather(openai_tasks + ollama_tasks)
        
        if task_results:
            valid_conversations = [conv for conv in task_results if conv is not None and conv is not TypeError]
            if valid_conversations:
                valid_conversations.sort(key=lambda x: x["chunk_index"])
                
                actual_indices = [conv["chunk_index"] for conv in valid_conversations]
                if actual_indices != sorted(actual_indices):
                    logger.warning(f"[PDF {index}] El orden de las conversaciones no coincide con el esperado")
                
                with open(jsonl_file, "w", encoding="utf-8") as f:
                    output_str = "\n".join(json.dumps({"messages": conv["messages"], "topic": conv["topic"]}, ensure_ascii=False) 
                                        for conv in valid_conversations)
                    f.write(output_str + "\n")
                    logger.info(f"[PDF {index}] Dataset conversacional generado y guardado en JSONL")
            else:
                logger.warning(f"[PDF {index}] No se generaron conversaciones")
    except Exception as e:
        logger.error(f"[PDF {index}] Error al procesar el PDF: {e}")

In [57]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import asyncio
import os
from pathlib import Path
import math


def process_pdf_wrapper(args):
    index, pdf_path, output_dir = args
    logger = logging.getLogger()
    logger.info(f"[PDF {index}] Inicio del procesamiento del PDF: {Path(pdf_path).name}")
    try:
        asyncio.run(process_pdf(index, pdf_path, output_dir))
        logger.info(f"[PDF {index}] Procesamiento del PDF completado")
    except Exception as e:
        logger.error(f"[PDF {index}] Error durante el procesamiento: {str(e)}")
        raise

def generate(pdf_files,max_workers=12):
    output_dir = "outputs_2"
    output_folder_path = Path(output_dir)
    os.makedirs(output_folder_path, exist_ok=True)
    
    logger = logging.getLogger()
    logger.info(f"Generando Datasets - max_workers={max_workers}, # Files: {len(pdf_files)}")
    
    # Dividir archivos en lotes para optimizar el uso de memoria
    batch_size = max(1, math.ceil(len(pdf_files) / max_workers))
    batches = [pdf_files[i:i + batch_size] for i in range(0, len(pdf_files), batch_size)]
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for batch_idx, batch in enumerate(batches):
            logger.info(f"Procesando lote {batch_idx + 1}/{len(batches)} con {len(batch)} archivos")
            tasks = [(i + batch_idx * batch_size, p, output_dir) for i, p in enumerate(batch)]
            for _ in tqdm(
                executor.map(process_pdf_wrapper, tasks),
                total=len(batch),
                desc=f"Batch {batch_idx + 1}/{len(batches)}"
            ):
                pass
            logger.info(f"Lote {batch_idx + 1}/{len(batches)} completado")
    logger.info("Generación de datasets finalizada")

In [ ]:
%%time
files = get_files()
generate(files,max_workers=15)

2025-08-12 14:02:03,329 - INFO - Generando Datasets - max_workers=15, # Files: 89
2025-08-12 14:02:03,333 - INFO - Procesando lote 1/15 con 6 archivos
Batch 1/15:   0%|          | 0/6 [00:00<?, ?it/s]2025-08-12 14:02:03,452 - INFO - [PDF 0] Inicio del procesamiento del PDF: L1_XVII_cap_IV.pdf
2025-08-12 14:02:03,452 - INFO - [PDF 1] Inicio del procesamiento del PDF: L1_IX_cap_III.pdf
2025-08-12 14:02:03,453 - INFO - [PDF 2] Inicio del procesamiento del PDF: manual-para-la-gestion-de-riesgos-lavados-de-activos-y-financiacion-del-terrorismo (1).pdf
2025-08-12 14:02:03,453 - INFO - [PDF 3] Inicio del procesamiento del PDF: L1_XVII_cap_III.pdf
2025-08-12 14:02:03,453 - INFO - [PDF 4] Inicio del procesamiento del PDF: Modelo_de_Administracion_del_Riesgo_de_LAFT_y_Contrabando_web.pdf
2025-08-12 14:02:03,454 - INFO - [PDF 5] Inicio del procesamiento del PDF: manual-para-la-gestion-de-riesgos-lavados-de-activos-y-financiacion-del-terrorismo.pdf
2025-08-12 14:02:03,458 - INFO - [PDF 0] Procesan

2025-08-12 14:02:10,659 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:02:10,662 - INFO - Retrying request to /responses in 0.445769 seconds
2025-08-12 14:02:10,921 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:02:10,924 - INFO - Retrying request to /responses in 0.448642 seconds
2025-08-12 14:02:11,469 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:02:11,472 - INFO - Retrying request to /responses in 0.393622 seconds
2025-08-12 14:02:12,767 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:02:12,771 - INFO - Retrying request to /responses in 0.398085 seconds
2025-08-12 14:02:14,682 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:02:16,450 - INFO - HTTP Request: POST https://api.openai.com

Rate limit hit for gpt-4o-mini: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-1SttmZMm9hEuBw7UWMxZimku on tokens per min (TPM): Limit 200000, Used 200000, Requested 5712. Please try again in 1.713s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


2025-08-12 14:02:23,051 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:02:23,134 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:02:23,296 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:02:23,664 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:02:23,668 - INFO - Retrying request to /responses in 0.384902 seconds
2025-08-12 14:02:23,778 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"


Rate limit hit for gpt-4o-mini: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-1SttmZMm9hEuBw7UWMxZimku on tokens per min (TPM): Limit 200000, Used 200000, Requested 5741. Please try again in 1.722s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


...

2025-08-12 14:05:48,398 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:05:49,148 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:05:50,480 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"


Rate limit hit for gpt-4o-mini: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-1SttmZMm9hEuBw7UWMxZimku on tokens per min (TPM): Limit 200000, Used 200000, Requested 5723. Please try again in 1.716s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


2025-08-12 14:05:50,958 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:05:51,565 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:05:51,569 - INFO - Retrying request to /responses in 0.458713 seconds
2025-08-12 14:05:53,033 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:05:53,307 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-08-12 14:05:53,311 - INFO - Retrying request to /responses in 0.410521 seconds
2025-08-12 14:05:53,860 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:05:55,832 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:05:58,007 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:05:59,457 - INFO - HTTP Request: POST https://api.open

Error generating conversation with OpenAI gpt-4o-mini: 1 validation error for Conversation
  Invalid JSON: EOF while parsing a string at line 9 column 1567 [type=json_invalid, input_value='{\n  "messages": [\n    ...ntecedentes del cliente', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid


2025-08-12 14:16:40,594 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:16:43,853 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:16:46,621 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:16:49,024 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:16:51,716 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:16:52,256 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:16:52,481 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:16:53,798 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-12 14:16:54,295 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-08-12 14:16:57,569 - INFO - HTTP Request: POST https://api.op

### Saving

In [24]:
%pip install --upgrade datasets huggingface_hub ipywidgets pandas


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./outputs/data/*.jsonl")

dataset


/workspace/fine-tunning-models/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 4682 examples [00:00, 17601.63 examples/s]


DatasetDict({
    train: Dataset({
        features: ['messages', 'topic'],
        num_rows: 4682
    })
})

In [4]:
import os
hf_token = os.environ.get("HUGGING_FACE_KEY")
dataset.push_to_hub("jeanmcm/b_risks",token=hf_token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 55.02ba/s]
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

Processing Files (0 / 1)                :  14%|█▎        |  524kB / 3.86MB, 1.31MB/s  
Processing Files (0 / 1)                :  27%|██▋       | 1.05MB / 3.86MB, 1.75MB/s  
Processing Files (0 / 1)                :  95%|█████████▌| 3.67MB / 3.86MB, 4.59MB/s  

Processing Files (1 / 1)                : 100%|██████████| 3.86MB / 3.86MB, 3.21MB/s  

Processing Files (1 / 1)                : 100%|██████████| 3.86MB / 3.86MB, 2.76MB/s  
New Data Upload                         : 100%|██████████| 3.86MB / 3.86MB, 2.76MB/s  
                                        : 100%|██████████| 3.86MB / 3.86MB            
Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.87s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jeanmcm/b_risks/commit/6f625bb43e6b588893a70d8769613ed11130179a', commit_message='Upload dataset', commit_description='', oid='6f625bb43e6b588893a70d8769613ed11130179a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jeanmcm/b_risks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jeanmcm/b_risks'), pr_revision=None, pr_num=None)

# Testing RAG with Open WebUI 

In [ ]:
import requests
import json
import sys
import time
from IPython.display import display, clear_output, HTML

url = "https://vwlppjjfa98c9x-8080.proxy.runpod.net"
api_key ="sk-05568562f28844fe997cadf960a346cd"

messages =  [{"role": "user", "content": "Que es el Riesgo Financiero?"}]

try:
    # Realizar la solicitud con stream=True
    with requests.post(f"{url}/api/chat/completions", stream=True,headers={
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    },json={
      "model":"bosft-riesgos-rag-model",
      "messages":messages,
      "stream":True
      }) as response:
        response.raise_for_status()
        # Variable para almacenar la salida acumulada
        accumulated_output = ""

        # Iterar sobre las líneas de la respuesta
        for line in response.iter_lines():
            if line:
                # Decodificar la línea
                decoded_line = line.decode('utf-8').strip()
                # Si la línea comienza con "data:", extraer el contenido
                if decoded_line.startswith("data:"):
                    decoded_line = decoded_line[5:].strip()  # Quitar "data: "

                # Ignorar líneas vacías o marcadores de fin como "[DONE]"
                if not decoded_line or decoded_line == "[DONE]" :
                    continue
                
                
                try:
                    # Parsear si es JSON
                    data = json.loads(decoded_line)
                    if "choices" not in data: continue
                    
                    delta = data['choices'][0]['delta']
                    if "content" in delta: new_data = delta['content']
                except json.JSONDecodeError:
                    # Si no es JSON, usar la línea como texto
                    new_data = decoded_line

                # Acumular y mostrar la salida dinámicamente
                if new_data:
                    accumulated_output += new_data
                    # Limpiar la salida anterior y mostrar la nueva
                    clear_output(wait=True)
                    display(HTML(f"<b>Respuesta en streaming:</b> {accumulated_output}"))
                    time.sleep(0.1)  # Pequeña pausa para visibilidad
                    

except requests.exceptions.RequestException as e:
    print(f"\nError en la solicitud: {e}")

# Testing with Flowise